In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import cv2
import mediapipe as mp
from matplotlib import pyplot as plt
from math import cos, sin
import os

In [2]:
path="AFLW2000"
files_names_with_extention=os.listdir(path)
files_names_duplicated=[x.split('.')[0] for x in files_names_with_extention]

from collections import OrderedDict 
names_list = list(OrderedDict.fromkeys(files_names_duplicated))
names_list.pop(0)

'Code'

In [3]:
len(names_list)

2000

In [4]:
img_array=[]
col_mat=["pitch", "yaw", "roll", "h", "v","t", "scale"]
mat_array=np.zeros((2000,7))
for i, name in enumerate (names_list):
    imgbgr = cv2.imread("AFLW2000/"+name+'.jpg')
    img = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
    img_array.append(img)
    mat = sio.loadmat("AFLW2000/"+name+'.mat')
    mat_array[i,:] = mat['Pose_Para'][0]
    
df_mat=pd.DataFrame(mat_array, columns=col_mat)


In [5]:
cols_names_x=[]
cols_names_y=[]
for i in range(0,468):
    cols_names_x.append('x'+(str(i+1)))
for i in range(0,468):
    cols_names_y.append('y'+(str(i+1)))  
df_x=pd.DataFrame(columns=cols_names_x)
df_y=pd.DataFrame(columns=cols_names_y)
cols=cols_names_x+cols_names_y

data_x=np.zeros((2000,468))
data_y=np.zeros((2000,468))

ex=[]

In [6]:
mp_face_mesh=mp.solutions.face_mesh.FaceMesh(static_image_mode=True,max_num_faces=2)
for j in range(2000):
    image=img_array[j]
    #Face Mesh
    result=mp_face_mesh.process(image)
    height,width,_=image.shape 
    if result.multi_face_landmarks!=None and len(result.multi_face_landmarks)==1: 
        for facial_landmarks in result.multi_face_landmarks:
            for i in range(0,468):
                pt1=facial_landmarks.landmark[i]
                data_x[j,i]=pt1.x *width
                data_y[j,i]=pt1.y *height
    else:
          ex.append(j)
df_x=pd.DataFrame(data_x,columns=cols_names_x)
df_y=pd.DataFrame(data_y,columns=cols_names_y)

In [7]:
len(ex)

165

In [8]:
df_x=pd.DataFrame(data_x,columns=cols_names_x)
df_y=pd.DataFrame(data_y,columns=cols_names_y)
#recenter data
df_x=df_x.sub(df_x['x2'], axis=0)
df_y=df_y.sub(df_y['y2'], axis=0)
#concat x df with y df for all data
df=pd.concat([df_x,df_y],axis=1)
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,-1.594493,0.0,0.273813,-4.258785,0.613944,1.997763,6.299825,-56.482413,8.745705,9.618656,...,0.619939,-1.925826,-1.350760,2.052802,2.130935,-69.563305,-66.108902,-62.707633,-75.773491,-79.072487
1,10.903019,0.0,14.204845,8.424470,-0.682220,4.278231,21.117206,21.212491,25.584948,24.658419,...,3.412548,1.508716,7.802358,3.647751,5.678961,-38.382760,-36.237073,-34.802011,-41.403008,-43.588780
2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,15.480316,0.0,8.992133,-11.289753,-3.494462,-5.583005,-8.172262,-48.574658,-14.342710,-18.661697,...,-2.699080,-6.357291,-9.659413,-0.900123,-0.725886,-68.829855,-64.488271,-60.867530,-89.236236,-94.180499
4,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
#join features with corresponding poses
tabel=pd.concat([df,df_mat],axis=1)
tabel.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y466,y467,y468,pitch,yaw,roll,h,v,t,scale
0,-1.594493,0.0,0.273813,-4.258785,0.613944,1.997763,6.299825,-56.482413,8.745705,9.618656,...,-62.707633,-75.773491,-79.072487,-0.399231,0.018227,0.085676,231.465317,262.066589,-102.793602,0.001587
1,10.903019,0.0,14.204845,8.424470,-0.682220,4.278231,21.117206,21.212491,25.584948,24.658419,...,-34.802011,-41.403008,-43.588780,0.470065,1.189533,0.300959,330.819702,173.194244,-44.367504,0.001181
2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.184650,0.881137,-0.236852,301.050323,200.051437,-48.590317,0.001041
3,15.480316,0.0,8.992133,-11.289753,-3.494462,-5.583005,-8.172262,-48.574658,-14.342710,-18.661697,...,-60.867530,-89.236236,-94.180499,-0.175379,0.299208,-0.373374,262.805054,211.201294,-102.680016,0.001294
4,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.882169,1.198004,-1.033374,323.511200,218.552780,-21.119057,0.001247


In [10]:
pdname=pd.DataFrame(names_list,columns=["file_name"])
pdname.head()

,file_name
0,image00002
1,image00004
2,image00006
3,image00008
4,image00010


In [11]:
data=pd.concat([pdname,tabel],axis=1)
data.head()

,file_name,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,y466,y467,y468,pitch,yaw,roll,h,v,t,scale
0,image00002,-1.594493,0.0,0.273813,-4.258785,0.613944,1.997763,6.299825,-56.482413,8.745705,...,-62.707633,-75.773491,-79.072487,-0.399231,0.018227,0.085676,231.465317,262.066589,-102.793602,0.001587
1,image00004,10.903019,0.0,14.204845,8.424470,-0.682220,4.278231,21.117206,21.212491,25.584948,...,-34.802011,-41.403008,-43.588780,0.470065,1.189533,0.300959,330.819702,173.194244,-44.367504,0.001181
2,image00006,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.184650,0.881137,-0.236852,301.050323,200.051437,-48.590317,0.001041
3,image00008,15.480316,0.0,8.992133,-11.289753,-3.494462,-5.583005,-8.172262,-48.574658,-14.342710,...,-60.867530,-89.236236,-94.180499,-0.175379,0.299208,-0.373374,262.805054,211.201294,-102.680016,0.001294
4,image00010,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.882169,1.198004,-1.033374,323.511200,218.552780,-21.119057,0.001247


In [12]:
#filter zero featurs rows
data=data[np.count_nonzero(data.values, axis=1) > len(df.columns)-2]

In [13]:
data.head()

,file_name,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,y466,y467,y468,pitch,yaw,roll,h,v,t,scale
0,image00002,-1.594493,0.0,0.273813,-4.258785,0.613944,1.997763,6.299825,-56.482413,8.745705,...,-62.707633,-75.773491,-79.072487,-0.399231,0.018227,0.085676,231.465317,262.066589,-102.793602,0.001587
1,image00004,10.903019,0.0,14.204845,8.424470,-0.682220,4.278231,21.117206,21.212491,25.584948,...,-34.802011,-41.403008,-43.588780,0.470065,1.189533,0.300959,330.819702,173.194244,-44.367504,0.001181
3,image00008,15.480316,0.0,8.992133,-11.289753,-3.494462,-5.583005,-8.172262,-48.574658,-14.342710,...,-60.867530,-89.236236,-94.180499,-0.175379,0.299208,-0.373374,262.805054,211.201294,-102.680016,0.001294
5,image00013,7.879311,0.0,3.162163,-11.412464,-2.037916,-4.110177,-8.307070,-60.646449,-10.829245,...,-50.452325,-68.443993,-70.299277,-0.026812,0.011965,-0.220662,225.624603,199.878860,-109.525414,0.001357
6,image00014,3.662235,0.0,2.879432,-5.797742,-0.535919,-0.180472,1.884145,-49.433380,2.246135,...,-43.464111,-50.942150,-52.917141,0.057119,0.110732,-0.043283,239.685089,190.156235,-109.077309,0.001341


In [14]:
#save data to csv
data.to_csv("data2.csv",index=False)

In [15]:
#ensure data retrival
d=pd.read_csv("data2.csv")
d.head()

,file_name,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,y466,y467,y468,pitch,yaw,roll,h,v,t,scale
0,image00002,-1.594493,0.0,0.273813,-4.258785,0.613944,1.997763,6.299825,-56.482413,8.745705,...,-62.707633,-75.773491,-79.072487,-0.399231,0.018227,0.085676,231.465317,262.066589,-102.793602,0.001587
1,image00004,10.903019,0.0,14.204845,8.424470,-0.682220,4.278231,21.117206,21.212491,25.584948,...,-34.802011,-41.403008,-43.588780,0.470065,1.189533,0.300959,330.819702,173.194244,-44.367504,0.001181
2,image00008,15.480316,0.0,8.992133,-11.289753,-3.494462,-5.583005,-8.172262,-48.574658,-14.342710,...,-60.867530,-89.236236,-94.180499,-0.175379,0.299208,-0.373374,262.805054,211.201294,-102.680016,0.001294
3,image00013,7.879311,0.0,3.162163,-11.412464,-2.037916,-4.110177,-8.307070,-60.646449,-10.829245,...,-50.452325,-68.443993,-70.299277,-0.026812,0.011965,-0.220662,225.624603,199.878860,-109.525414,0.001357
4,image00014,3.662235,0.0,2.879432,-5.797742,-0.535919,-0.180472,1.884145,-49.433380,2.246135,...,-43.464111,-50.942150,-52.917141,0.057119,0.110732,-0.043283,239.685089,190.156235,-109.077309,0.001341


In [16]:
d.shape

(1835, 944)